In [1]:
import numpy as np
import pandas as pd

import os
from joblib import dump, load, Parallel, delayed

from spsd_utils import distance_matrix, exponent_kernel
from lapl_utils import produce_laplacian_datasets, produce_normlaplacian_datasets
from system_utils import computation_time, send_message
from loader_utils import load_data_oftypes

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.cross_validation import train_test_split

from scipy.linalg import svd, eigh, eigvalsh, eigvals

from IPython.display import clear_output

import matplotlib
matplotlib.use("Pdf")
clear_output()

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 200

colors = ['#0000ff', '#00ff00', '#ff0000', '#ffff00']
markers = ['o'] * 4 + ['x'] * 4

clear_output()

# 1. Define functions

In [3]:
class EigenValuesTransformer:

    def __init__(self, k=0):
        self.k = k

    def _transform(self, cov):
        U, S, V = svd(cov)
        for k in range(self.k):
            S[-k] = 0.
        return np.dot(np.dot(U, np.diag(S)), V)

    def fit(self, X, y=None):
        # doesn't do anything
        pass

    def fit_transform(self, X, y=None):
        return self.transform(X)

    def transform(self, X):
#         X = np.array(X)
        return np.array([self._transform(cov) for cov in X])

In [4]:
def precompute_distances(data, metric='spsd', output_name=None, 
                         rank_decrease=None, return_res=True):
    if not os.path.exists('dump/distances'):
        os.mkdir('dump/distances')
    
    if output_name is not None:
        if os.path.exists('dump/distances/%s' % output_name):
            return load('dump/distances/%s' % output_name)
        
    distances = {}
    if rank_decrease is not None:
        data = EigenValuesTransformer(k=rank_decrease).transform(data)

    distances = distance_matrix(data, metric)

    if output_name is not None:
        dump(distances, 'dump/distances/%s' % output_name)
    if return_res:
        return distances

# 2. Load data

In [26]:
dtypes = ('AD', 'Normal', 'EMCI', 'LMCI')
orig_matrices, inv_matrices, target, idx, diagnosis, names = load_data_oftypes(('AD', 'Normal'))
data_raw = produce_laplacian_datasets(orig_matrices, inv_matrices)
data = data_raw['original_nonnormed']

In [16]:
orig_matrices, inv_matrices, target, idx, diagnosis, names = load_data_oftypes(dtypes)
data_raw = produce_laplacian_datasets(orig_matrices, inv_matrices)
data = data_raw['original_nonnormed']

## 2.1 Precompute distances

In [7]:
Parallel(n_jobs=-1)(delayed(precompute_distances)(data, output_name='original_nonnormed_drop_%d' % k, rank_decrease=k, return_res=False) for k in np.arange(1,15))
clear_output()

# 3. Look how distance matrix changes while lowering rank

In [8]:
# @computation_time
def distance_matrix_diff(data, rank_decrease_list, save=True, show=False, output_name=None):
    diff_matrix = {rank_decrease: None for rank_decrease in rank_decrease_list}
    max_diffs = []
    
    no_drop = precompute_distances(data, output_name=output_name)
    send_message('Precomputed original distance matrix for %s' % output_name)
    
    for rank_decrease in rank_decrease_list:
        drop = precompute_distances(data, rank_decrease=rank_decrease, 
                                    output_name='%s_drop_%d' % (output_name, rank_decrease) if output_name is not None else None)
        
        send_message('Precomputed distance matrix for %s with %d drop' % (output_name, rank_decrease))
        
        diff_matrix[rank_decrease] = np.array([[np.abs(no_drop[i,j] - drop[i,j]) 
                                                for i in range(drop.shape[0])] 
                                                for j in range(drop.shape[1])]) / np.max([np.abs(no_drop),
                                                                                             np.abs(drop)])
        max_diffs.append(np.max(diff_matrix[rank_decrease]))
        
    max_diff = np.max(max_diffs)
    for rank_decrease in rank_decrease_list:
        if save:
            if not os.path.exists('dump/plots'):
                os.mkdir('dump/plots')
            fig = plt.Figure()
            sns.heatmap(diff_matrix[rank_decrease], vmax=max_diff, xticklabels=False, yticklabels=False,
                                   cmap='RdBu_r')
            plt.savefig('dump/plots/diff_drop_%d.png' % rank_decrease)
            plt.close()
        if show:
            fig = plt.Figure()
            sns.heatmap(diff_matrix[rank_decrease], vmax=max_diff, xticklabels=False, yticklabels=False,
                                   cmap='RdBu_r')
            plt.close()

In [15]:
try:
    distance_matrix_diff(data, np.arange(2, 14, 2), output_name='original_nonnormed')
    send_message('Done!')
except Exception as e:
    send_message('Something went wrong!\n\n%s' % e)

In [10]:
!ffmpeg -start_number 2 -r 2 -pattern_type glob -i 'dump/plots/diff_drop_*.png'  distance_matrix_break_2.gif
# !ffmpeg -i distance_matrix_break.mp4 -r 2 -pix_fmt rgb24 -s qcif distance_matrix_break.gif

ffmpeg version 3.3.1 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavcodec     57. 89.100 / 57. 89.100
  libavformat    57. 71.100 / 57. 71.100
  libavdevice    57.  6.100 / 57.  6.100
  libavfilter     6. 82.100 /  6. 82.100
  libavresample   3.  5.  0 /  3.  5.  0
  libswscale      4.  6.100 /  4.  6.100
  lib